# Perplexity Sonar

## API Settings

In [1]:
import os
from dotenv import load_dotenv

api_key = os.environ.get("SONAR_API_KEY")
load_dotenv()

True

In [2]:
url = "https://api.perplexity.ai/chat/completions"
headers = {
    "accept": "application/json",
    "authorization": f"Bearer {api_key}",
    "content-type": "application/json"
}

## Payload

In [3]:
# Normal
payload = {
    "model": "sonar",
    "messages": [
        {"role": "system", "content":"""You are a helpful AI assistant.
Your task is to review all retrieved search results and synthesize them into a single, coherent answer.
Rules:
1. Base your answer only on the provided search results.
2. Merge overlapping information and resolve any contradictions using the most credible sources.
3. Present the answer in a clear, well-structured format with concise language.
4. If relevant, include key facts, dates, figures, and context to support the explanation.
5. If search results do not fully answer the question, explicitly state the missing information.
"""},
        {"role": "user", "content": "What is dark vessel detection?"}
    ],
    "stream": False,
}

# Academic
# payload = {
#     "model": "sonar",
#     "messages": [{"role": "user", "content": "What is dark vessel detection?"}],
#     "stream": False,
#     "search_mode": "academic",
#     "search_after_date_filter": "8/1/2023",
#     "web_search_options": {"search_context_size": "high"},
#     # "search_domain_filter": [
#     #     "ieee.org",
#     # ]
# }

## Search

In [ ]:
import requests
import json

response = requests.post(url, headers=headers, json=payload).json()
print(f"Response:\n{response['choices'][0]['message']['content']}")

# citations = response.get("citations", [])
# print(f"\nTotal {len(citations)} Citations:")
# for citation in citations:
#     print(citation)

search_results = response.get("search_results", [])
print(f"\nTotal {len(search_results)} Search Results:")
for search_result in search_results:
    print(json.dumps(search_result, indent=2, ensure_ascii=False))

Response:
**Dark vessel detection** is a method of identifying vessels at sea that do not broadcast their location through standard identification systems like the Automatic Identification System (AIS), making them "dark" or invisible to conventional tracking methods[4]. These vessels often evade surveillance for purposes such as illegal fishing, trafficking, or other illicit activities.

This detection is primarily achieved using **satellite technology** that scans large ocean areas routinely, employing multiple sensor types:

- **Optical imagers** capture images in visible light.
- **Synthetic Aperture Radar (SAR)** penetrates clouds and darkness, providing reliable imaging day and night.
- **Radio Frequency (RF) geolocation** detects radio transmissions from vessels[1].

The process involves analyzing satellite data with advanced algorithms and feature detection to identify anomalies indicative of vessels. Detected vessels are then cross-checked against AIS and Vessel Monitoring Sys

In [9]:
for search_result in search_results:
    print(search_result)

{'title': 'Satellite dark vessel detection for maritime domain ...', 'url': 'https://www.starboardintelligence.com/articles/satellite-dark-vessel-detection-for-maritime-domain-awareness', 'date': '2022-11-12', 'last_updated': '2025-06-16'}
{'title': 'Machine-learning system can detect dark vessels faster', 'url': 'https://www.esa.int/Enabling_Support/Space_Engineering_Technology/Shaping_the_Future/Machine-learning_system_can_detect_dark_vessels_faster', 'date': '2024-10-16', 'last_updated': '2024-10-31'}
{'title': 'Dark Vessel Detection', 'url': 'https://mda.space/dark-vessel', 'date': '2025-07-18', 'last_updated': '2025-07-18'}
{'title': 'Dark Vessels', 'url': 'https://globalfishingwatch.org/research-project-dark-vessels/', 'date': '2022-05-03', 'last_updated': '2024-10-30'}
{'title': 'Dark vessels and how to find them', 'url': 'https://canadiangeographic.ca/articles/dark-vessels-and-how-to-find-them/', 'date': '2021-04-22', 'last_updated': '2024-12-02'}


In [5]:
# save search result as json file
import json

with open("result/response.md", "w", encoding="utf-8") as f:
    f.write(response['choices'][0]['message']['content'])

with open("result/search_results.json", "w", encoding="utf-8") as f:
    json.dump(search_results, f, ensure_ascii=False, indent=2)

# Crawl4ai

## Crawling

In [15]:
import shutil
import os
import nest_asyncio

shutil.rmtree("result/md_docs")
os.makedirs("result/md_docs", exist_ok=True)
nest_asyncio.apply()

In [ ]:
import asyncio
from crawl4ai import AsyncWebCrawler, BrowserConfig, CrawlerRunConfig, CacheMode
import aiofiles
import re

async def crawl_single_url(crawler, url, title):
    try:
        run_conf = CrawlerRunConfig(cache_mode=CacheMode.BYPASS)
        result = await crawler.arun(url=url, config=run_conf)
        
        if result.success:
            # 處理檔名
            safe_filename = re.sub(r'[^\w\s-]', '', title).strip()[:100]
            safe_filename = re.sub(r'[-\s]+', '-', safe_filename)
            filename = f"result/md_docs/{safe_filename}.md"
            
            # 使用 aiofiles 進行異步文件寫入
            async with aiofiles.open(filename, "w", encoding="utf-8") as f:
                await f.write(result.markdown.raw_markdown)
            
            return {"status": "success", "title": title, "url": url}
        else:
            return {"status": "failed", "title": title, "url": url, "error": result.error_message}
    except Exception as e:
        return {"status": "error", "title": title, "url": url, "error": str(e)}

async def parallel_crawling(search_results):
    browser_conf = BrowserConfig(headless=True)
    
    async with AsyncWebCrawler(config=browser_conf) as crawler:
        # 建立所有爬取任務
        tasks = [
            crawl_single_url(crawler, sr['url'], sr['title']) 
            for sr in search_results
        ]
        
        # 並行執行所有任務
        results = await asyncio.gather(*tasks, return_exceptions=True)
        
        # 處理結果
        for result in results:
            if isinstance(result, dict):
                if result["status"] == "success":
                    print(f"✅ Saved: {result['title']}")
                else:
                    print(f"❌ Failed: {result['title']} - {result.get('error', 'Unknown error')}")
            else:
                print(f"❌ Exception: {result}")

asyncio.run(parallel_crawling(search_results))

[INIT].... → Crawl4AI 0.7.3 

[FETCH]... ↓ https://canadiangeographic.ca/articles/dark-vessels-and-how-to-find-them/                            |
✓ | ⏱: 1.39s 

[SCRAPE].. ◆ https://canadiangeographic.ca/articles/dark-vessels-and-how-to-find-them/                            |
✓ | ⏱: 0.00s 

[COMPLETE] ● https://canadiangeographic.ca/articles/dark-vessels-and-how-to-find-them/                            |
✓ | ⏱: 1.40s 

[FETCH]... ↓ https://www.esa.int/Enabling_Support/Space_Engin...e-learning_system_can_detect_dark_vessels_faster  |
✓ | ⏱: 1.58s 

...

[COMPLETE] ● https://globalfishingwatch.org/research-project-dark-vessels/                                        |
✓ | ⏱: 2.54s 

[FETCH]... ↓ https://mda.space/dark-vessel                                                                        |
✓ | ⏱: 3.05s 

[SCRAPE].. ◆ https://mda.space/dark-vessel                                                                        |
✓ | ⏱: 0.01s 

[COMPLETE] ● https://mda.space/dark-vessel                                                                        |
✓ | ⏱: 3.06s 

✅ Saved: Satellite dark vessel detection for maritime domain ...
✅ Saved: Machine-learning system can detect dark vessels faster
✅ Saved: Dark Vessel Detection
✅ Saved: Dark Vessels
✅ Saved: Dark vessels and how to find them


# LLM

In [13]:
research_response_content = """
**Dark vessel detection** generally refers to identifying vessels (usually ships) that are deliberately not transmitting their location signals (such as AIS or GPS) and therefore appear "dark" or invisible to conventional maritime tracking systems. This detection is often done using satellite imagery combined with advanced techniques like deep learning to automatically identify ships engaged in activities like illicit transshipment or smuggling.

Specifically, one study describes training a convolutional neural network on satellite images to detect various types of cargo vessels and ship-to-ship transfers, including "dark" transfers—where ships are not broadcasting their presence—to uncover potentially illicit activities[1]. This approach uses remotely sensed imagery to automatically detect vessels that evade standard tracking by GPS-based systems, revealing over 400 such dark transshipment events in a conflict-prone region between Ukraine and Russia since 2022.

In summary, **dark vessel detection** is a remote sensing and machine learning method for identifying ships that are deliberately avoiding detection by normal tracking technologies, primarily using satellite imagery to enhance maritime domain awareness and combat illegal activities at sea.
"""

In [14]:
system_prompt = """
You are a follow-up question generator. Given the user's provided information, produce 5 clear, specific follow-up questions to gather additional key facts or perspectives.

Guidelines:
1. Do not repeat known information.
2. Ensure each question is clear, easy to understand, and aims for a specific or quantifiable answer.
3. Focus on filling research gaps, validating hypotheses, or revealing emerging trends.
4. Move from key gaps → background context → comparisons/changes → details/impacts.
5. Avoid leading or biased wording; remain neutral and open-ended.
6. Exclude illegal, privacy-violating, or sensitive personal data.

Output format:
List exactly 5 numbered questions.  
Example:
1. <First question>  
2. <Second question>  
3. <Third question>  
4. <Fourth question>  
5. <Fifth question>
"""

In [15]:
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()

client = OpenAI()

# 呼叫 Chat API
response = client.chat.completions.create(
    model="gpt-4.1-nano",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": research_response_content}
    ]
)
    
# 取得回覆
question_response_content = response.choices[0].message.content
print(question_response_content)

1. What are the most common types of satellite imagery (e.g., optical, synthetic aperture radar) used in dark vessel detection, and how do their detection capabilities compare?  
2. How accurate are current deep learning models in distinguishing dark vessels from other objects or natural features in satellite images?  
3. What specific features or patterns do models analyze to identify ships engaged in dark activities, and how consistent are these features across different regions?  
4. How has the frequency of detected dark vessel activities changed over time in the conflict-prone areas, and what factors might explain these trends?  
5. What challenges or limitations are currently faced in automating dark vessel detection, particularly regarding false positives or environmental conditions?


In [18]:
import re

questions = re.findall(r'^\d+\.\s*(.+)', question_response_content, flags=re.MULTILINE)
for question in questions:
    print(question)

What are the most common types of satellite imagery (e.g., optical, synthetic aperture radar) used in dark vessel detection, and how do their detection capabilities compare?  
How accurate are current deep learning models in distinguishing dark vessels from other objects or natural features in satellite images?  
What specific features or patterns do models analyze to identify ships engaged in dark activities, and how consistent are these features across different regions?  
How has the frequency of detected dark vessel activities changed over time in the conflict-prone areas, and what factors might explain these trends?  
What challenges or limitations are currently faced in automating dark vessel detection, particularly regarding false positives or environmental conditions?


In [ ]:
import nest_asyncio
from research import single_research

nest_asyncio.apply()

print(f"Question:\n{questions[0]}")
single_research(questions[0])

Question:
What are the most common types of satellite imagery (e.g., optical, synthetic aperture radar) used in dark vessel detection, and how do their detection capabilities compare?  

Response:
The most common types of satellite imagery used in dark vessel detection are **optical imagery** and **Synthetic Aperture Radar (SAR) imagery**. Their detection capabilities complement each other, especially in challenging conditions:

- **Optical imagery** captures visible light and can provide high-resolution, detailed vessel images, suitable for identifying vessel types and activities under clear weather and daylight conditions. Studies show deep learning methods applied to optical satellite data (e.g., PlanetScope, Sentinel-2) can achieve high accuracy in detecting vessels and even ship-to-ship transfers[1][4][9]. However, optical imaging is limited by weather (e.g., cloud cover) and darkness.

- **SAR imagery** uses radar signals to image surfaces and penetrates clouds and darkness, prov

[INIT].... → Crawl4AI 0.7.3 

[FETCH]... ↓ https://arc.aiaa.org/doi/10.2514/6.2022-2110                                                         |
✓ | ⏱: 2.35s 

[SCRAPE].. ◆ https://arc.aiaa.org/doi/10.2514/6.2022-2110                                                         |
✓ | ⏱: 0.04s 

[COMPLETE] ● https://arc.aiaa.org/doi/10.2514/6.2022-2110                                                         |
✓ | ⏱: 2.40s 

...

[COMPLETE] ● https://ieeexplore.ieee.org/document/9884642/                                                        |
✓ | ⏱: 6.69s 

[FETCH]... ↓ https://ieeexplore.ieee.org/document/10282146/                                                       |
✓ | ⏱: 6.67s 

[SCRAPE].. ◆ https://ieeexplore.ieee.org/document/10282146/                                                       |
✓ | ⏱: 0.02s 

[COMPLETE] ● https://ieeexplore.ieee.org/document/10282146/                                                       |
✓ | ⏱: 6.69s 

✅ Saved: Automatic Detection of Dark Ship-to-Ship Transfers Using Deep Learning and Satellite Imagery
✅ Saved: Multi-Temporal Sentinel-1 SAR Images For Dark Vessel Detection And Classification Using a Circlenet Model
✅ Saved: Vessel Traffic Density Maps Based on Vessel Detection in Satellite Imagery
✅ Saved: AI-Driven Marine Vessel Detection Through Satellite Imagery: A Deep Learning Approach
✅ Saved: Insight into the Collocation of Multi-Source Satellite Imagery for Multi-Scale Vessel Detection
✅ Saved: A Hybrid Method for Vessel Detection in High-Resolution Satellite Imagery
✅ Saved: Assessment of Maritime Vessel Detection and Tracking Using Integrated SAR Imagery and AIS/V-Pass Data
✅ Saved: Vessel Detection in Satellite Images using Deep Learning
✅ Saved: Self-Supervised Vessel Detection from Low Resolution Satellite Imagery
✅ Saved: Land use land cover mapping and snow cover detection in Himalayan region using machine learning and multispectral Sentinel-2 satellite imagery

Total 